<a href="https://colab.research.google.com/github/Progressive-Programmer/image_to_ttf/blob/main/image_to_ttf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import cv2
import numpy as np
import os

def process_font_grid_v2(image_path, rows=7, cols=6, target_size=512):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 21, 10)

    h_img, w_img = thresh.shape
    cell_h, cell_w = h_img // rows, w_img // cols

    output_folder = "svg_glyphs_input" # New folder for clean PNGs
    os.makedirs(output_folder, exist_ok=True)

    # 10% margin to definitely kill the grid lines
    margin_y = int(cell_h * 0.10)
    margin_x = int(cell_w * 0.10)

    char_idx = 0
    for r in range(rows):
        for c in range(cols):
            y1, y2 = r * cell_h + margin_y, (r + 1) * cell_h - margin_y
            x1, x2 = c * cell_w + margin_x, (c + 1) * cell_w - margin_x
            cell = thresh[y1:y2, x1:x2]

            cnts, _ = cv2.findContours(cell, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if cnts:
                x_c, y_c = [], []
                for cnt in cnts:
                    if cv2.contourArea(cnt) > 100: # Higher threshold to ignore tiny noise
                        bx, by, bw, bh = cv2.boundingRect(cnt)
                        x_c.extend([bx, bx + bw]); y_c.extend([by, by + bh])

                if x_c:
                    # Tight crop
                    char_crop = cell[min(y_c):max(y_c), min(x_c):max(x_c)]

                    # Normalization with 20% PADDING (prevents cut-offs)
                    h_c, w_c = char_crop.shape
                    inner_size = int(target_size * 0.6) # Smaller inner size
                    scale = inner_size / max(h_c, w_c)
                    nw, nh = int(w_c * scale), int(h_c * scale)
                    resized = cv2.resize(char_crop, (nw, nh), interpolation=cv2.INTER_AREA)

                    canvas = np.zeros((target_size, target_size), dtype=np.uint8)
                    x_off, y_off = (target_size - nw) // 2, (target_size - nh) // 2 # Center both ways
                    canvas[y_off:y_off+nh, x_off:x_off+nw] = resized

                    cv2.imwrite(f"{output_folder}/glyph_{char_idx:03d}.png", canvas)
                    char_idx += 1
    print(f"Cleaned {char_idx} glyphs with extra padding.")

process_font_grid_v2('input_image.jpeg')

Cleaned 42 glyphs with extra padding.


In [19]:
!apt-get update
!apt-get install -y potrace fontforge

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (3,405 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
potrace is already the newest version (1.16-2).
fo

In [20]:
import os
import subprocess
import cv2
import numpy as np

# --- STEP A: Vectorize PNGs to SVGs ---
input_png_folder = "svg_glyphs_input" # The folder from the V2 Cleaning Script
svg_folder = "svg_final"
os.makedirs(svg_folder, exist_ok=True)

png_files = sorted([f for f in os.listdir(input_png_folder) if f.endswith('.png')])
print(f"Vectorizing {len(png_files)} files...")

for filename in png_files:
    name = os.path.splitext(filename)[0]
    bmp_path = os.path.join(svg_folder, f"{name}.bmp")
    svg_path = os.path.join(svg_folder, f"{name}.svg")

    # Convert PNG to BMP for Potrace
    img = cv2.imread(os.path.join(input_png_folder, filename), cv2.IMREAD_GRAYSCALE)
    cv2.imwrite(bmp_path, img)

    # Trace to SVG
    subprocess.run(["potrace", "--svg", "--flat", bmp_path, "-o", svg_path])
    os.remove(bmp_path)

# --- STEP B: Build the Font File ---
fontforge_script = f"""
import fontforge
import os

font = fontforge.font()
font.fontname = "MyCustomFontFixed"
font.fullname = "MyCustomFont Fixed"
font.familyname = "MyCustomFont"

svg_dir = "{svg_folder}"
svg_files = sorted([f for f in os.listdir(svg_dir) if f.endswith('.svg')])
target_chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789"

for i, filename in enumerate(svg_files):
    if i >= len(target_chars): break

    glyph = font.createChar(ord(target_chars[i]))
    glyph.importOutlines(os.path.join(svg_dir, filename))

    # FIX CUT-OFFS: Scale the glyph to fit the 'Em' square properly
    # This shrinks the character slightly and centers it so it doesn't hit edges
    glyph.transform([0.8, 0, 0, 0.8, 0, 0])

    # Auto-align the character so it sits on the baseline
    bbox = glyph.boundingBox()
    glyph.transform([1, 0, 0, 1, 0, -bbox[1]])

    # Set spacing so letters aren't squashed together
    glyph.left_side_bearing = 60
    glyph.right_side_bearing = 60
    glyph.width = 800

font.generate("MyCustomFont_Fixed.ttf")
print("TTF File Created Successfully")
"""

with open("build_font_final.py", "w") as f:
    f.write(fontforge_script)

# Execute the script
!fontforge -script build_font_final.py

# --- STEP C: Verification ---
if os.path.exists("MyCustomFont_Fixed1.ttf"):
    print("\n✅ SUCCESS: 'MyCustomFont_Fixed.ttf' is ready!")
    print("Download it from the file sidebar.")
else:
    print("\n❌ FAILED: The .ttf was not generated. Check if 'svg_glyphs_input' has images.")

Vectorizing 42 files...
Copyright (c) 2000-2024. See AUTHORS for Contributors.
 License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>
 with many parts BSD <http://fontforge.org/license.html>. Please read LICENSE.
 Version: 20201107
 Based on sources from 2024-06-24 13:55 UTC-ML-D-GDK3.
PythonUI_Init()
copyUIMethodsToBaseTable()
TTF File Created Successfully

❌ FAILED: The .ttf was not generated. Check if 'svg_glyphs_input' has images.
